<a href="https://colab.research.google.com/github/pkrc267/IndoorLocalization/blob/master/Combined_Prediction_06_04_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import pandas.util.testing as tm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn import datasets
from io import StringIO

from sklearn.tree import export_graphviz
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn import metrics

In [0]:
frame0=pd.read_csv("/content/newRssiDblock-3.csv")

In [0]:
frame0.head()

,Name,Address,Signal Level,X,Y,Z,MAC,Location
0,BPGC-NAB,70:EA:1A:22:8F:E0,-41,10,16,10,124150763130848,101610
1,BPGC-NAB,08:4F:A9:10:4B:80,-49,10,16,10,9138231856000,101610
2,BPGC-NAB,00:A3:8E:60:F2:A0,-72,10,16,10,702468387488,101610
3,BPGC-WIFI,40:01:7A:B1:00:40,-70,10,16,10,70375097565248,101610
4,BPGC-NAB,70:EA:1A:22:A9:A0,-66,10,16,10,124150763137440,101610


In [0]:
frame = pd.get_dummies(frame0)

In [0]:
frame.head()

,Signal Level,X,Y,Z,MAC,Location,Name_BPGC-HOSTEL,Name_BPGC-LT,Name_BPGC-NAB,Name_BPGC-WIFI,Name_CRT-MIFI,Name_DIRECT-1D-HP DeskJet 2600 series,Name_SAIMAA,Name_TG,Name_TheLastUchiha,Name_bhikkhu,Name_sks,Address_00:A3:8E:4E:29:A0,Address_00:A3:8E:60:F2:A0,Address_00:A3:8E:67:76:40,Address_04:95:E6:82:45:88,Address_06:92:26:A2:85:2C,Address_08:4F:A9:10:4B:80,Address_08:4F:A9:2D:77:40,Address_08:4F:A9:2D:8A:40,Address_08:4F:A9:2D:8A:A0,Address_08:4F:A9:30:00:00,Address_08:4F:A9:39:68:20,Address_08:4F:A9:39:68:40,Address_08:4F:A9:39:6A:40,Address_08:4F:A9:43:38:A0,Address_40:01:7A:01:B8:80,Address_40:01:7A:01:B8:8F,Address_40:01:7A:AF:D3:60,Address_40:01:7A:AF:D3:6F,Address_40:01:7A:B1:00:40,Address_40:01:7A:B1:00:41,Address_40:01:7A:C6:A5:60,Address_40:01:7A:C6:A5:61,Address_40:01:7A:CA:3F:A0,Address_40:01:7A:CA:3F:AE,Address_40:01:7A:CA:3F:AF,Address_40:01:7A:CC:88:20,Address_40:01:7A:CC:88:2F,Address_60:6D:C7:A3:30:E3,Address_68:3B:78:E9:C0:E0,Address_70:EA:1A:22:87:60,Address_70:EA:1A:22:8F:E0,Address_70:EA:1A:22:92:40,Address_70:EA:1A:22:96:00,Address_70:EA:1A:22:96:20,Address_70:EA:1A:22:96:A0,Address_70:EA:1A:22:97:80,Address_70:EA:1A:22:98:20,Address_70:EA:1A:22:9B:80,Address_70:EA:1A:22:9B:E0,Address_70:EA:1A:22:9F:20,Address_70:EA:1A:22:9F:80,Address_70:EA:1A:22:9F:E0,Address_70:EA:1A:22:A0:20,Address_70:EA:1A:22:A5:A0,Address_70:EA:1A:22:A9:40,Address_70:EA:1A:22:A9:A0,Address_70:EA:1A:22:AC:00,Address_80:14:A8:BE:BB:28,Address_88:63:DF:BB:74:A5,Address_8C:21:A7:E1:4E:22,Address_A8:B4:56:CA:8F:40,Address_A8:B4:56:CC:DE:00,Address_AC:E2:D3:B6:BA:1E,Address_BC:16:F5:9F:6E:F0
0,-41,10,16,10,124150763130848,101610,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,-49,10,16,10,9138231856000,101610,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,-72,10,16,10,702468387488,101610,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,-70,10,16,10,70375097565248,101610,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,-66,10,16,10,124150763137440,101610,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [0]:
from sklearn.ensemble import RandomForestClassifier

In [0]:
# feature_cols=['MAC','Signal Level']
# feature_cols=['X','Y','Z']
TV=frame.drop(['X','Y'], axis=1)
FV=frame.Location
TV.fillna(TV.mean, inplace=True)

In [0]:
TV.head()

,Signal Level,Z,MAC,Location,Name_BPGC-HOSTEL,Name_BPGC-LT,Name_BPGC-NAB,Name_BPGC-WIFI,Name_CRT-MIFI,Name_DIRECT-1D-HP DeskJet 2600 series,Name_SAIMAA,Name_TG,Name_TheLastUchiha,Name_bhikkhu,Name_sks,Address_00:A3:8E:4E:29:A0,Address_00:A3:8E:60:F2:A0,Address_00:A3:8E:67:76:40,Address_04:95:E6:82:45:88,Address_06:92:26:A2:85:2C,Address_08:4F:A9:10:4B:80,Address_08:4F:A9:2D:77:40,Address_08:4F:A9:2D:8A:40,Address_08:4F:A9:2D:8A:A0,Address_08:4F:A9:30:00:00,Address_08:4F:A9:39:68:20,Address_08:4F:A9:39:68:40,Address_08:4F:A9:39:6A:40,Address_08:4F:A9:43:38:A0,Address_40:01:7A:01:B8:80,Address_40:01:7A:01:B8:8F,Address_40:01:7A:AF:D3:60,Address_40:01:7A:AF:D3:6F,Address_40:01:7A:B1:00:40,Address_40:01:7A:B1:00:41,Address_40:01:7A:C6:A5:60,Address_40:01:7A:C6:A5:61,Address_40:01:7A:CA:3F:A0,Address_40:01:7A:CA:3F:AE,Address_40:01:7A:CA:3F:AF,Address_40:01:7A:CC:88:20,Address_40:01:7A:CC:88:2F,Address_60:6D:C7:A3:30:E3,Address_68:3B:78:E9:C0:E0,Address_70:EA:1A:22:87:60,Address_70:EA:1A:22:8F:E0,Address_70:EA:1A:22:92:40,Address_70:EA:1A:22:96:00,Address_70:EA:1A:22:96:20,Address_70:EA:1A:22:96:A0,Address_70:EA:1A:22:97:80,Address_70:EA:1A:22:98:20,Address_70:EA:1A:22:9B:80,Address_70:EA:1A:22:9B:E0,Address_70:EA:1A:22:9F:20,Address_70:EA:1A:22:9F:80,Address_70:EA:1A:22:9F:E0,Address_70:EA:1A:22:A0:20,Address_70:EA:1A:22:A5:A0,Address_70:EA:1A:22:A9:40,Address_70:EA:1A:22:A9:A0,Address_70:EA:1A:22:AC:00,Address_80:14:A8:BE:BB:28,Address_88:63:DF:BB:74:A5,Address_8C:21:A7:E1:4E:22,Address_A8:B4:56:CA:8F:40,Address_A8:B4:56:CC:DE:00,Address_AC:E2:D3:B6:BA:1E,Address_BC:16:F5:9F:6E:F0
0,-41,10,124150763130848,101610,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,-49,10,9138231856000,101610,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,-72,10,702468387488,101610,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,-70,10,70375097565248,101610,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,-66,10,124150763137440,101610,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [0]:
TV_train,TV_test,FV_train,FV_test=train_test_split(TV, FV, test_size=0.1, random_state=1)

In [0]:
clf = RandomForestClassifier(max_depth=8,random_state=0)
clf.fit(TV_train,FV_train)
FV_pred = clf.predict(TV_test)
print(clf.score(TV_test,FV_test)*100)

18.823529411764707


In [0]:
# FV_test.to_numpy()

In [0]:
original = pd.Series([],dtype='float64')
original['x'] = FV_test.to_numpy(copy=True)
original['y'] = FV_test.to_numpy(copy=True)
# original['z'] = FV_test

In [0]:
# original['x'].shape
# type(original)

In [0]:
v = FV_test.to_numpy(copy=True)

In [0]:
# v.size

In [0]:
# original['x']
# FV_test.size

In [0]:
for i in range(0, FV_test.size):
  # original['z'][i] = v[i] %100
  v[i] = v[i] // 100
  original['y'][i] = v[i] %100
  v[i] = v[i] // 100
  original['x'][i] = v[i] %100
  v[i] = v[i] // 100

In [0]:
# original['x'].shape

In [0]:
predicted = pd.Series([],dtype='float64')
predicted['x'] = FV_pred.copy()
predicted['y'] = FV_pred.copy()
# predicted['z'] = FV_pred

In [0]:
for i in range(0,FV_pred.size):
  # predicted['z'][i] = FV_pred[i] %100
  FV_pred[i] = FV_pred[i] // 100
  predicted['y'][i] = FV_pred[i] %100
  FV_pred[i] = FV_pred[i] // 100
  predicted['x'][i] = FV_pred[i] %100
  FV_pred[i] = FV_pred[i] // 100

In [0]:
# predicted['x'].shape

In [0]:
# type(original['x'])

In [0]:
def relaxaCheck(predicted, original):
  relaxed = original.copy()
  for i in range(0, predicted.size):
    if predicted[i] in range(original[i]-2, original[i]+2):
      relaxed[i] = 1
    else:
      relaxed[i] = 0
  
  return relaxed

In [0]:
relaxed_x = relaxaCheck(predicted['x'], original['x'])#.to_numpy())
relaxed_y = relaxaCheck(predicted['y'], original['y'])#.to_numpy())
# relaxed_z = relaxaCheck(predicted['z'], original['z'].to_numpy())

In [0]:
# relaxed_x

In [0]:
c=0
for i in range(0, relaxed_x.size):
  if relaxed_x[i] == relaxed_y[i] == 1: #relaxed_z[i] == 1:
    c+=1

print(100*c/relaxed_x.size)

61.76470588235294
